In [1]:
#Библиотеки для работы с данными
import numpy as np
from numpy import asarray
import csv
from copy import deepcopy
#Для построения и работы с графиками
import matplotlib.pyplot as plt
%matplotlib inline
#Для работы с моделями
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet_v2 import preprocess_input
import keras
import tensorflow as tf
#Для работы с изображениями
import cv2
from PIL import Image
#Для работы с метрикой f1-score
import tensorflow_addons as tfa
f1 =tfa.metrics.F1Score(num_classes=9, average='weighted')

C:\Users\sazsa\anaconda3\Lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [2]:
#Загрузим модели из чекпойнтов
model_acc = tf.keras.models.load_model("D:/Skillbox/diploma_Ml_mid/checkpoints/emotion_recog/acc_2blocks.h5", compile=False)
model_f1 = tf.keras.models.load_model("D:/Skillbox/diploma_Ml_mid/checkpoints/emotion_recog/f1_2blocks.h5", compile=False)

In [3]:
#Создадим объект класса ImageDataGenerator (для подачи в модели):
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  preprocessing_function=preprocess_input)

In [4]:
# Названия классов:
class_names = {0: 'anger',
     1: 'contempt',
     2: 'disgust',
     3: 'fear',
     4: 'happy',
     5: 'neutral',
     6: 'sad',
     7: 'surprise',
     8: 'uncertain'}

Попробуем всё объединить и проверить на работоспособность:

In [5]:
try:
    required_size = (224, 224)

    vid=cv2.VideoCapture(0)
    vid.set(3, 448)  # Set width
    vid.set(4, 224)  # Set height

    if not (vid.isOpened()):
        print("Could not open video device")


    while True:

        # Разделяем видео на отдельные кадры-фреймы
        ret, frame = vid.read()


        # инициализируем детектор
        face_detector = cv2.FaceDetectorYN_create('D:/Skillbox/diploma_Ml_mid/data/YuNet/face_detection_yunet_2023mar.onnx',
                          "", 
                          (640, 480),
                          score_threshold=0.5)
    
        faces = face_detector.detect(frame)[1]
    
    
        if faces is not None:
            faces = faces.astype(int)
            face = faces[0]
            x1, y1, f_width, f_height = face[0], face[1], face[2], face[3]
            x2, y2 = x1 + f_width, y1 + f_height


            # Следим, чтобы бокс не вылез за пределы экрана, иначе функция вылетит с ошибкой:
            if x2 > 640:
                x2 = 640
            if y2 > 480:
                y2 = 480
        
        
            # Нарисуем бокс вокруг лица
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    
            # обрезка изображения до рамки лица для подачи в модель
            face_boundary = frame[y1:y2, x1:x2]
            face_boundary = Image.fromarray(face_boundary)
    
        
            # Приведем к нужному размеру для подачи в модель     
            face_image = face_boundary.resize(required_size)
            face_array = np.asarray(face_image,'float32')
            face_array = face_array[None, ...]
            
            
            #Подаем в модель
            test_generator = test_datagen.flow(face_array,
                                       batch_size = 1,
                                       seed =12,
                                       shuffle  = False)

            # Подставим в модели и сделаем предсказание
            predict_result_acc = model_acc.predict(test_generator)
            predict_result_f1 = model_f1.predict(test_generator)
            model_predictions = np.array([predict_result_acc, predict_result_f1])
            weights=[0.91803279, 0.08196721]
            predictions = np.tensordot(model_predictions, weights, axes=((0),(0)))
            pred = np.argmax(predictions)
            pred_class = class_names[pred]
        
            #Проверка работоспособности
            print(pred_class)
            # Подписи класса
            # шрифт
            font = cv2.FONT_HERSHEY_SIMPLEX
            # размер шрифта
            fontScale = 0.6
            # Толщина линии (пикселей)
            thickness = 1
            cv2.putText(frame, f'{pred_class}', (x1 + 5, y1 - 5), font, fontScale, (0, 255, 0), thickness, cv2.LINE_AA)
    
        else:
            print('No faces detected!')
            cv2.putText(frame, 'No faces detected!', (50, 50), font, fontScale, (0, 255, 0), thickness, cv2.LINE_AA)


        # Вывод результата
        cv2.imshow('camera',frame)

   
        # Нажмите на 'q' чтобы выйти:
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
            print("[EXIT] Camera stopped")

except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("[EXIT] Camera stopped")
    # Destroy all the windows
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 417ms/step
neutral
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 54ms/step
surprise
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 54ms/step
surprise
1/1 [==============================] - 0s 58ms/step
surprise
1/1 [==============================] - 0s 53ms/step
surprise
No faces detected!
No faces detected!
1/1 [==============================] - 0s 55ms/step
neutral
1/1 [==============================] - 0s 53ms/step
happy
1/1 [==============================] - 0s 53ms/step
surprise
1/1 [==============================] - 0s 54ms/step
surprise
1/1 [==============================] - 0s 57ms/step
surprise
1/1 [==============================] - 0s 54ms/step

В задании к работе четко указано, что модель необходимо обернуть в Python-класс с определенным набором методов. Единственное, что не совсем понятно: весь ли код должен быть обернут в класс или только непосредственная часть для предсказания (загрузка весов предобученной модели распознавания эмоций, собственно предсказание по изображению и его, предсказания, вывод). Попробуем имплементировать всё. Для начала создадим более узкий класс для предсказания. Назовем его EmoRecog.

In [2]:
class EmoRecog():
    # Названия классов:
    em_names = {0: 'anger',
         1: 'contempt',
         2: 'disgust',
         3: 'fear',
         4: 'happy',
         5: 'neutral',
         6: 'sad',
         7: 'surprise',
         8: 'uncertain'}
    
    def __init__(self):
         self.model_acc = tf.keras.models.load_model("D:/Skillbox/diploma_Ml_mid/checkpoints/emotion_recog/acc_2blocks.h5", compile=False)
         self.model_f1 = tf.keras.models.load_model("D:/Skillbox/diploma_Ml_mid/checkpoints/emotion_recog/f1_2blocks.h5", compile=False)
         #Создадим объект класса ImageDataGenerator (для подачи в модели):
         self.test_datagen = ImageDataGenerator(rescale = 1./255, preprocessing_function=preprocess_input)
    def predict(self, img):
        #Подаем в модель
        test_generator = self.test_datagen.flow(img, batch_size = 1, seed =12, shuffle  = False)
        #Получаем предсказания:
        acc_preds = self.model_acc.predict(test_generator)
        f1_preds = self.model_f1.predict(test_generator)
        model_preds = np.array([acc_preds, f1_preds])
        model_preds = np.tensordot(model_preds, [0.91803279, 0.08196721], axes=((0),(0)))
        model_preds = EmoRecog.em_names[np.argmax(model_preds)]
        return model_preds

In [4]:
  # инициализируем детектор
face_detector = cv2.FaceDetectorYN_create('D:/Skillbox/diploma_Ml_mid/data/YuNet/face_detection_yunet_2023mar.onnx',
                          "", 
                          # (im_width, im_height),
                          (640, 480),                
                          score_threshold=0.5)

Проверка работоспособности данного набора кода:

In [5]:
rec_model = EmoRecog()

try:
    required_size = (224, 224)
    vid=cv2.VideoCapture(0)
    vid.set(3, 448)  # Set width
    vid.set(4, 224)  # Set height

    if not (vid.isOpened()):
        print("Could not open video device")

    while True:

        # Разделяем видео на отдельные кадры-фреймы
        ret, frame = vid.read()


        faces = face_detector.detect(frame)[1]
    
    
        if faces is not None:
            faces = faces.astype(int)
            face = faces[0]
            x1, y1, f_width, f_height = face[0], face[1], face[2], face[3]
            x2, y2 = x1 + f_width, y1 + f_height


            # Следим, чтобы бокс не вылез за пределы экрана, иначе функция вылетит с ошибкой:
            if x2 > 640:
                x2 = 640
            if y2 > 480:
                y2 = 480
        
            # Нарисуем бокс вокруг лица
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
    
            # обрезка изображения до рамки лица для подачи в модель
            face_boundary = frame[y1:y2, x1:x2]
            face_boundary = Image.fromarray(face_boundary)

           
            # Приведем к нужному размеру для подачи в модель     
            face_image = face_boundary.resize(required_size)
            face_array = np.asarray(face_image,'float32')
            face_array = face_array[None, ...]
            
            
            # #Подаем в модель

            pred_class = rec_model.predict(face_array)
            #Проверка работоспособности
            print(pred_class)
            # Подписи класса
            # шрифт
            font = cv2.FONT_HERSHEY_SIMPLEX
            # размер шрифта
            fontScale = 0.6
            # Толщина линии (пикселей)
            thickness = 1
            cv2.putText(frame, f'{pred_class}', (x1 + 5, y1 - 5), font, fontScale, (0, 255, 0), thickness, cv2.LINE_AA)
    
        else:
            print('No faces detected!')
            cv2.putText(frame, 'No faces detected!', (50, 50), font, fontScale, (0, 255, 0), thickness, cv2.LINE_AA)


        # Вывод результата
        cv2.imshow('camera',frame)

   
        # Нажмите на 'q' чтобы выйти:
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
            print("[EXIT] Camera stopped")

except KeyboardInterrupt:
    # Release the Video Device
    vid.release()
    # Message to be displayed after releasing the device
    print("[EXIT] Camera stopped")
    # Destroy all the windows
    cv2.destroyAllWindows()

1/1 [==============================] - 0s 409ms/step
anger
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 54ms/step
fear
1/1 [==============================] - 0s 54ms/step
fear
1/1 [==============================] - 0s 54ms/step
fear
1/1 [==============================] - 0s 54ms/step
fear
1/1 [==============================] - 0s 54ms/step
surprise
1/1 [==============================] - 0s 66ms/step
anger
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 56ms/step
anger
1/1 [==============================] - 0s 56ms/step
anger
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 53ms/step
anger
1/1 [==============================] - 0s 58ms/step
anger
1/1 [==============================] - 0s 55ms/step
anger
1/1 [==============================] - 0s 53ms/step
happy
1/1 [==============================] - 0s 53ms/step
happy
1/1 [=========

Exception ignored in: <function UniquePtr.__del__ at 0x00000242A47A8360>
Traceback (most recent call last):
  File "C:\Users\sazsa\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\framework\c_api_util.py", line 71, in __del__
    obj = self._obj
          ^^^^^^^^^
AttributeError: 'ScopedTFGraph' object has no attribute '_obj'


[EXIT] Camera stopped


А вот теперь напишем класс, который охватывает практически всё оставшееся от разделения изображения на кадры до вывода предсказаний на экран.

In [4]:
class VideoDisplay():
    #Шрифт
    font = cv2.FONT_HERSHEY_SIMPLEX
    #Размер шрифта
    fontScale = 0.6
    # Толщина линии (пикселей)
    thickness = 1
    #Цвет (зеленый)
    font_color = (0, 255, 0)
    #Параметры фрейма:
    xmax=640 
    ymax=480
    #Для изменения размеров далее
    required_size = (224, 224)
    
    def __init__(self):
        self.vid = cv2.VideoCapture(0)
        self.vid.set(3, 448)  # установка ширины дисплея
        self.vid.set(4, 224)  # установка высоты дисплея
        #Проверка подключения камеры:
        if not (self.vid.isOpened()):
            print("Could not open video device")
        self.face_detection_model = cv2.FaceDetectorYN_create('D:/Skillbox/diploma_Ml_mid/data/YuNet/face_detection_yunet_2023mar.onnx',
                          "", 
                          (self.xmax, self.ymax),                
                          score_threshold=0.5)

    def __del__(self):
        # Отсановим камеру и выведем сообщение об остановке
        self.vid.release()
        print("[EXIT] Camera stopped")

    def putText (self, frame, text, x,y):
        cv2.putText(frame, text, (x, y), self.font, self.fontScale, self.font_color, self.thickness, cv2.LINE_AA)
        cv2.imshow('camera', frame)

    def noface_error(self):
        #Если лица не находятся, выводим сообщение на кадр:
        #Напечатаем текст для проверки
        print('No faces detected!')
        self.putText('No faces detected!', 50, 50)

    def face_to_array(self, frame, face_boundary):
        face_boundary = Image.fromarray(face_boundary)
        face_image = face_boundary.resize(self.required_size)
        face_array = np.asarray(face_image,'float32')
        face_array = face_array[None, ...]
        return face_array
        

    def pred_emotion(self, model):
        # Разделяем видео на отдельные кадры-фреймы
        ret, frame = self.vid.read()
        #Находим лица
        faces = self.face_detection_model.detect(frame)[1]


        if faces is not None:
            faces = faces.astype(int)
            face = faces[0]
            x1, y1, f_width, f_height = face[0], face[1], face[2], face[3]
            x2, y2 = x1 + f_width, y1 + f_height


            # Следим, чтобы бокс не вылез за пределы экрана, иначе функция вылетит с ошибкой:
            if x2 > self.xmax:
                x2 = self.xmax
            if y2 > self.ymax:
                y2 = self.ymax
        
            # Нарисуем бокс вокруг лица
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            face_boundary = frame[y1:y2, x1:x2]
            # обрезка изображения до рамки лица для подачи в модель
            face_array = self.face_to_array(frame, face_boundary)

             # Подаем в модель
            pred_class = model.predict(face_array)
            #Проверка работоспособности
            print(pred_class)
            self.putText(frame, f'{pred_class}', x1 + 5, y1 - 5)
        
        else:
            print('No faces detected!')
            self.putText(frame, 'No faces detected!',50, 50)
        
        return cv2.imshow('camera',frame)

Итоговая проверка кода:

In [5]:
rec_model = EmoRecog()

try:

    disp = VideoDisplay()

    while True:
        disp.pred_emotion(rec_model)
        # Нажмите на 'q' чтобы выйти:
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
            print("[EXIT] Camera stopped")

except KeyboardInterrupt:
    #Остановка камеры:
    del disp
    
#Закрытие всех окон:
cv2.destroyAllWindows()

1/1 [==============================] - 0s 413ms/step
anger
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 53ms/step
anger
1/1 [==============================] - 0s 58ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 53ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 53ms/step
neutral
No faces detected!
No faces detected!
No faces detected!
No faces detected!
No faces detected!
1/1 [==============================] - 0s 52ms/step
neutral
1/1 [==============================] - 0s 53ms/step
neutral
1/1 [==============================] - 0s 52ms/step
neutral
1/1 [==============================] - 0s 59ms/step
neutral
1/1 [==============================] - 0s 55ms/step
neutral
1/1 [=================

Замечательно, похоже, что всё работает и очевидных ошибок и вылетов кода нет. Камера успешно инициируется и закрывается по завершению работы. Модель предсказания эмоций так же не ругается и не вылетает.

Попробуем сделать так, чтобы наш класс мог обработать сразу несколько лиц на кадре.

In [3]:
class VideoDisplay():
    #Шрифт
    font = cv2.FONT_HERSHEY_SIMPLEX
    #Размер шрифта
    fontScale = 0.6
    # Толщина линии (пикселей)
    thickness = 1
    #Цвет (зеленый)
    font_color = (0, 255, 0)
    #Параметры фрейма:
    xmax=640 
    ymax=480
    #Для изменения размеров далее
    required_size = (224, 224)
    
    def __init__(self):
        self.vid = cv2.VideoCapture(0)
        self.vid.set(3, 448)  # установка ширины дисплея
        self.vid.set(4, 224)  # установка высоты дисплея
        #Проверка подключения камеры:
        if not (self.vid.isOpened()):
            print("Could not open video device")
        self.face_detection_model = cv2.FaceDetectorYN_create('D:/Skillbox/diploma_Ml_mid/data/YuNet/face_detection_yunet_2023mar.onnx',
                          "", 
                          (self.xmax, self.ymax),                
                          score_threshold=0.5)

    def __del__(self):
        # Отсановим камеру и выведем сообщение об остановке
        self.vid.release()
        print("[EXIT] Camera stopped")

    def putText (self, frame, text, x,y):
        cv2.putText(frame, text, (x, y), self.font, self.fontScale, self.font_color, self.thickness, cv2.LINE_AA)
        cv2.imshow('camera', frame)

    def noface_error(self):
        #Если лица не находятся, выводим сообщение на кадр:
        #Напечатаем текст для проверки
        print('No faces detected!')
        self.putText('No faces detected!', 50, 50)

    def face_to_array(self, frame, face_boundary):
        face_boundary = Image.fromarray(face_boundary)
        face_image = face_boundary.resize(self.required_size)
        face_array = np.asarray(face_image,'float32')
        face_array = face_array[None, ...]
        return face_array
        

    def pred_emotion(self, model):
        # Разделяем видео на отдельные кадры-фреймы
        ret, frame = self.vid.read()
        #Находим лица
        faces = self.face_detection_model.detect(frame)[1]


        if faces is not None:
            for face in faces:
                face = face.astype(int)
                x1, y1, f_width, f_height = face[0], face[1], face[2], face[3]
                x2, y2 = x1 + f_width, y1 + f_height


                # Следим, чтобы бокс не вылез за пределы экрана, иначе функция вылетит с ошибкой:
                if x2 > self.xmax:
                    x2 = self.xmax
                if y2 > self.ymax:
                    y2 = self.ymax
        
                # Нарисуем бокс вокруг лица
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                face_boundary = frame[y1:y2, x1:x2]
                # обрезка изображения до рамки лица для подачи в модель
                face_array = self.face_to_array(frame, face_boundary)

                 # Подаем в модель
                pred_class = model.predict(face_array)
                #Проверка работоспособности
                print(pred_class)
                self.putText(frame, f'{pred_class}', x1 + 5, y1 - 5)
        
        else:
            print('No faces detected!')
            self.putText(frame, 'No faces detected!',50, 50)
        
        return cv2.imshow('camera',frame)

In [4]:
rec_model = EmoRecog()

try:

    disp = VideoDisplay()

    while True:
        disp.pred_emotion(rec_model)
        # Нажмите на 'q' чтобы выйти:
        if cv2.waitKey(1) & 0xff == ord('q'):
            break
            print("[EXIT] Camera stopped")

except KeyboardInterrupt:
    #Остановка камеры:
    del disp
    
#Закрытие всех окон:
cv2.destroyAllWindows()

1/1 [==============================] - 0s 400ms/step
fear
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 51ms/step
neutral
1/1 [==============================] - 0s 53ms/step
happy
1/1 [==============================] - 0s 57ms/step
happy
1/1 [==============================] - 0s 51ms/step
happy
1/1 [==============================] - 0s 53ms/step
neutral
1/1 [==============================] - 0s 53ms/step
anger
1/1 [==============================] - 0s 54ms/step
anger
1/1 [==============================] - 0s 52ms/step
anger
1/1 [==============================] - 0s 61ms/step
anger
1/1 [==============================] - 0s 58ms/step
anger
1/1 [==============================] - 0s 57ms/step
anger
1/1 [==============================] - 0s 58ms/step
anger
1/1 [=====